In [13]:
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import pandas as pd

import helpers
import math
import pickle
import random

def make_train_inputs(input_seq, target_seq):
    inputs_, inputs_length_ = helpers.batch(input_seq)
    targets_, targets_length_ = helpers.batch(target_seq)
    
    return {
        'encoder_inputs:0': inputs_,
        'encoder_inputs_length:0': inputs_length_,
        'decoder_targets:0': targets_,
        'decoder_targets_length:0': targets_length_,
    }

def make_inference_inputs(input_seq):
    inputs_, inputs_length_ = helpers.batch(input_seq)

    return {
        'encoder_inputs:0': inputs_,
        'encoder_inputs_length:0': inputs_length_,
    }

def numericEncode(sent):

    if type(sent) == str: sent = sent.split(' ')
    
    return [vocab_dict[word] if word in vocab_dict else 2 for word in sent]

def decodeSent(sent):
    return [inv_map[i] for i in sent]

df_all = pd.read_pickle('processed_data\processed_data_v01_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl')
vocab_dict = pickle.load(open('dicts\word_dict_v01_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl', 'rb'))

#Encode sequences
df_all['alpha_Pair_1_encoding'] =  df_all['alpha_Pair_1_tokens'].apply(numericEncode)
df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(numericEncode)

df_all['Index'] = df_all.index.values

df_all_train = df_all.sample(frac=0.90, random_state=0)

df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

inv_map = {v: k for k, v in vocab_dict.items()}

dev_encoded_text = df_all_dev['alpha_Pair_0_encoding'].values
dev_decoded_text = df_all_dev['alpha_Pair_1_encoding'].values


PAD = 0
EOS = 1

vocab_size = len(vocab_dict) 
input_embedding_size = 256

length_from = 3
length_to = 49
vocab_lower = 0
vocab_upper = vocab_size
n_batch_size = 32

batches_in_epoch = df_all_train.shape[0]/n_batch_size

n_cells = 128
num_layers = 2

n_epochs = 1000

filename = 'seq2seq_enron_encode_128_decode_256_vocab_7239_embedding_256_seq_3_49_batch_32_layers_2_enkeep_10_dekeep_10-28679'
tf.reset_default_graph()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
session=tf.Session(config=tf.ConfigProto(
  allow_soft_placement=True, log_device_placement=True))    
#First let's load meta graph and restore weights
# We import the meta graph in the current default Graph
saver = tf.train.import_meta_graph('d:\coding\seq2seq\chkpt\\' + filename + '.meta')

# We restore the weights
saver.restore(session, 'd:\coding\seq2seq\chkpt\\' + filename)

# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
#x0 = graph.get_tensor_by_name('encoder_inputs:0')
#x1 = graph.get_tensor_by_name('encoder_inputs_length:0')
#x2 = graph.get_tensor_by_name('decoder_targets:0')
#x3 = graph.get_tensor_by_name('decoder_targets_length:0')


#Now, access the op that you want to run. 
#op_to_restore = graph.get_tensor_by_name('Adam:0')
op_to_restore1 = graph.get_tensor_by_name('Decoder/decoder_prediction_inference:0')
op_to_restore2 = graph.get_tensor_by_name('Decoder/decoder_prediction_prob_inference:0')

#print (session.run(op_to_restore, feed_dict))

result = session.run([op_to_restore1, op_to_restore2], \
                make_inference_inputs([numericEncode(df_all['alpha_Pair_0_tokens'].values[0])]))


session.close()

[n.name for n in tf.get_default_graph().as_graph_def().node if 'inference' in n.name]

df_all[['alpha_Pair_0_tokens', 'alpha_Pair_1_tokens']].to_csv('enron_token_pairs.csv', index=False, sep='|')

q = ['i', 'may', 'have', 'to', 'start', 'training', 'now', 'for', 'that', 'and', 'show', 'you', 'a', 'little', 'what', 'partying', 'drinking', 'and', 'going', 'all', 'out', 'is', 'all', 'about', '<EOS>']

' '.join(q)

#make a POST request
import requests
dictToSend = {'string':' '.join(q)}
res = requests.post('http://54.158.64.8:5000/api/predict', json=dictToSend)

print ('response from server:', res.text)

dictFromServer = res.json()

dictFromServer

with tf.Session(graph=tf.Graph()) as sess:
    _ = tf.contrib.seq2seq

    # We import the meta graph in the current default Graph
    saver = tf.train.import_meta_graph('d:\coding\seq2seq\chkpt\seq2seq_enron_encode_128_decode_256_vocab_7239_embedding_256_seq_3_49_batch_32_layers_2_enkeep_10_dekeep_10-12019.meta')

    # We restore the weights
    saver.restore(sess, 'd:\coding\seq2seq\chkpt\seq2seq_enron_encode_128_decode_256_vocab_7239_embedding_256_seq_3_49_batch_32_layers_2_enkeep_10_dekeep_10-12019')
    #[print(n.name) for n in tf.get_default_graph().as_graph_def().node]


#    # We use a built-in TF helper to export variables to constants
    output_graph_def = tf.graph_util.convert_variables_to_constants(
        sess, # The session is used to retrieve the weights
        tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
        ['Decoder/dynamic_rnn_decoder/Decoder/while/attention_decoder/Softmax', 
         'Decoder/dynamic_rnn_decoder_1/Decoder/while/attention_decoder_fn_inference/Softmax'])
                # The output node names are used to select the usefull nodes
 #   ) 
    
    # Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile('frozen_model.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())
    
    print("%d ops in the final graph." % len(output_graph_def.node))
    
  #  with tf.gfile.GFile('frozen_model.pb', "rb") as f:
        #graph_def = tf.GraphDef()
        #graph_def.ParseFromString(f.read())
        
    #with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        #tf.import_graph_def(graph_def)